# Aprendizaje Automático Cuántico (QML)

**Nombre:** Cristian David Araujo A.  
**Identificación:** 1089568350  
**Fecha:** 11/06/2025  


### Descripción del Código

El código implementa un clasificador variacional cuántico utilizando la librería PennyLane. A continuación, se describen las etapas principales del flujo de trabajo:

1. **Importación de Librerías**: Se importan las librerías necesarias, incluyendo `pennylane`, `numpy`, `sklearn`, y otras para el manejo de datos y métricas de evaluación.

2. **Definición de Parámetros y Configuración del Dispositivo**: 
    - Se define el número de qubits y capas del circuito cuántico.
    - Se configura un dispositivo cuántico simulado (`default.qubit`) con los qubits especificados.

3. **Feature Map**: Se define la función `statepreparation` para preparar el estado cuántico inicial basado en los datos de entrada.

4. **Ansatz**: 
    - En el primer modelo, se utiliza un Ansatz personalizado con capas definidas manualmente.
    - En el segundo modelo, se utiliza el Ansatz `BasicEntanglerLayers`.

5. **Circuito Cuántico**: Se define el circuito cuántico utilizando el Ansatz y el Feature Map. El circuito devuelve la expectativa de la puerta Pauli-Z en el primer qubit.

6. **Clasificador Variacional**: Se implementa la función `variational_classifier`, que combina el circuito cuántico con un sesgo para realizar predicciones.

7. **Funciones de Pérdida y Costo**: 
    - `square_loss`: Calcula la pérdida cuadrática entre las etiquetas reales y las predicciones.
    - `cost`: Evalúa el costo total del modelo basado en las predicciones y las etiquetas.

8. **Entrenamiento del Modelo**: 
    - En el primer modelo, se utiliza el optimizador `AdamOptimizer`.
    - En el segundo modelo, se utiliza el optimizador `SPSAOptimizer`.
    - Los pesos y sesgos del modelo se actualizan iterativamente para minimizar la función de costo.

9. **Evaluación del Modelo**: Se calculan métricas como `accuracy`, `precision`, `recall`, y `F1 Score` para evaluar el desempeño del clasificador cuántico en los datos de prueba.

10. **Comparativa de Modelos**: Se realiza una comparación entre dos modelos entrenados con diferentes configuraciones de Ansatz y optimizadores, destacando sus métricas y características.

Este flujo de trabajo combina técnicas de aprendizaje automático cuántico con métodos clásicos de preprocesamiento y evaluación de datos para resolver un problema de clasificación binaria.

In [7]:
# Importación de librerías necesarias
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math

# Definición de parámetros del circuito cuántico
num_qubits = 4
num_layers = 2

# Configuración del dispositivo cuántico
dev = qml.device("default.qubit", wires=num_qubits)

# Etapa del Feature Map: Preparación del estado cuántico
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

# Etapa del Ansatz: Definición de las capas del circuito cuántico
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

# Definición del circuito cuántico
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)
    for W in weights:
        layer(W)
    return qml.expval(qml.PauliZ(0))

# Función del clasificador variacional
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Etapa de Función de Pérdida: Definición de la función de pérdida
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

# Etapa de evaluación de la Función de Costo: Definición de la función de costo
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# Etapa de Análisis de Precisión: Definición de la función de precisión
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

# Preprocesamiento de los datos de entrada
df_train = pd.read_csv('train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

# División de datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived']
)

# Conversión de datos a formato compatible con PennyLane
X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

# Pesos iniciales definidos
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

# Configuración del optimizador
opt = AdamOptimizer(0.125)
num_it = 70
batch_size = math.floor(len(X_train) / num_it)

# Entrenamiento del clasificador cuántico
weights = weights_init
bias = bias_init
for it in range(num_it):
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Evaluación de precisión durante el entrenamiento
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )

# Pesos finales entrenados
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

# Evaluación de desempeño del clasificador cuántico final
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]
print("Accuracy:", accuracy_score(Y_test, predictions))
print("Precision:", precision_score(Y_test, predictions))
print("Recall:", recall_score(Y_test, predictions))
print("F1 Score:", f1_score(Y_test, predictions, average='macro'))

Iter:     1 | Cost: 2.3009054 | Accuracy: 0.3657928 
Iter:     2 | Cost: 2.0157333 | Accuracy: 0.3657928 
Iter:     3 | Cost: 1.6928554 | Accuracy: 0.3657928 
Iter:     4 | Cost: 1.4394783 | Accuracy: 0.3657928 
Iter:     5 | Cost: 1.2992964 | Accuracy: 0.5205993 
Iter:     6 | Cost: 1.2592430 | Accuracy: 0.6167291 
Iter:     7 | Cost: 1.2860673 | Accuracy: 0.6167291 
Iter:     8 | Cost: 1.3340117 | Accuracy: 0.6167291 
Iter:     9 | Cost: 1.3157367 | Accuracy: 0.6167291 
Iter:    10 | Cost: 1.2278186 | Accuracy: 0.6167291 
Iter:    11 | Cost: 1.1070846 | Accuracy: 0.6167291 
Iter:    12 | Cost: 1.0319251 | Accuracy: 0.6167291 
Iter:    13 | Cost: 0.9932298 | Accuracy: 0.6167291 
Iter:    14 | Cost: 0.9745684 | Accuracy: 0.6167291 
Iter:    15 | Cost: 0.9807690 | Accuracy: 0.7802747 
Iter:    16 | Cost: 0.9487227 | Accuracy: 0.7802747 
Iter:    17 | Cost: 0.9169272 | Accuracy: 0.7802747 
Iter:    18 | Cost: 0.8685417 | Accuracy: 0.7802747 
Iter:    19 | Cost: 0.8121101 | Accuracy: 0.78


### Actualización del Modelo

En esta etapa del flujo de trabajo, se ha realizado un cambio en la configuración del clasificador variacional cuántico:

- **Ansatz:** Se utiliza `BasicEntanglerLayers`, que aplica rotaciones parametrizadas y puertas CNOT en una configuración estándar.  
- **Optimizador:** Se emplea `SPSAOptimizer`.  
  

In [5]:
#
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import SPSAOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math

# Definición de parámetros del circuito cuántico
num_qubits = 4
num_layers = 2

# Configuración del dispositivo cuántico
dev = qml.device("default.qubit", wires=num_qubits)

# Etapa del Feature Map: Preparación del estado cuántico
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

# Etapa del Ansatz: Uso de BasicEntanglerLayers
# BasicEntanglerLayers aplica rotaciones de un parámetro en cada qubit seguidas de una cadena cerrada de puertas CNOT.
def ansatz(weights):
    qml.BasicEntanglerLayers(weights, wires=range(num_qubits))

# Definición del circuito cuántico
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)
    ansatz(weights)
    return qml.expval(qml.PauliZ(0))

# Función del clasificador variacional
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Etapa de Función de Pérdida: Definición de la función de pérdida
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

# Etapa de evaluación de la Función de Costo: Definición de la función de costo
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# Etapa de Análisis de Precisión: Definición de la función de precisión
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

# Preprocesamiento de los datos de entrada
df_train = pd.read_csv('train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

# División de datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived']
)

# Conversión de datos a formato compatible con PennyLane
X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

# Pesos iniciales definidos
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

# Configuración del optimizador SPSA
opt = SPSAOptimizer(maxiter=70)
num_it = 70# Importación de librerías necesarias
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import SPSAOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math

# Definición de parámetros del circuito cuántico
num_qubits = 4
num_layers = 2

# Etapa del Feature Map: Preparación del estado cuántico
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

# Etapa del Ansatz: Uso de BasicEntanglerLayers
# BasicEntanglerLayers aplica rotaciones de un parámetro en cada qubit seguidas de una cadena cerrada de puertas CNOT.
def ansatz(weights):
    qml.BasicEntanglerLayers(weights, wires=range(num_qubits))

# Definición del circuito cuántico
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)
    ansatz(weights)
    return qml.expval(qml.PauliZ(0))

# Función del clasificador variacional
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Etapa de Función de Pérdida: Definición de la función de pérdida
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

# Etapa de evaluación de la Función de Costo: Definición de la función de costo
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# Etapa de Análisis de Precisión: Definición de la función de precisión
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

# Preprocesamiento de los datos de entrada
df_train = pd.read_csv('train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

# División de datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived']
)

# Conversión de datos a formato compatible con PennyLane
X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

# Pesos iniciales definidos
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

# Configuración del optimizador SPSA
opt = SPSAOptimizer(maxiter=70)
num_it = 70
batch_size = math.floor(len(X_train) / num_it)

# Entrenamiento del clasificador cuántico
weights = weights_init
bias = bias_init
for it in range(num_it):
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias = opt.step(lambda w, b: cost(w, b, X_batch, Y_batch), weights, bias)

    # Evaluación de precisión durante el entrenamiento
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )

# Pesos finales entrenados
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

# Evaluación de desempeño del clasificador cuántico final
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]
print("Accuracy:", accuracy_score(Y_test, predictions))
print("Precision:", precision_score(Y_test, predictions))
print("Recall:", recall_score(Y_test, predictions))
print("F1 Score:", f1_score(Y_test, predictions, average='macro'))



Iter:     1 | Cost: 2.3967006 | Accuracy: 0.3657928 
Iter:     2 | Cost: 2.2644488 | Accuracy: 0.3657928 
Iter:     3 | Cost: 2.2534620 | Accuracy: 0.3657928 
Iter:     4 | Cost: 2.0162662 | Accuracy: 0.3657928 
Iter:     5 | Cost: 1.9804642 | Accuracy: 0.3657928 
Iter:     6 | Cost: 1.9779986 | Accuracy: 0.3657928 
Iter:     7 | Cost: 1.8367779 | Accuracy: 0.3657928 
Iter:     8 | Cost: 1.8588699 | Accuracy: 0.3657928 
Iter:     9 | Cost: 1.8501784 | Accuracy: 0.3657928 
Iter:    10 | Cost: 1.7616114 | Accuracy: 0.3657928 
Iter:    11 | Cost: 1.6894425 | Accuracy: 0.3657928 
Iter:    12 | Cost: 1.5969791 | Accuracy: 0.3657928 
Iter:    13 | Cost: 1.5764041 | Accuracy: 0.3657928 
Iter:    14 | Cost: 1.5656732 | Accuracy: 0.3657928 
Iter:    15 | Cost: 1.5527590 | Accuracy: 0.3657928 
Iter:    16 | Cost: 1.5480732 | Accuracy: 0.3657928 
Iter:    17 | Cost: 1.4375348 | Accuracy: 0.6167291 
Iter:    18 | Cost: 1.4402220 | Accuracy: 0.6167291 
Iter:    19 | Cost: 1.4349338 | Accuracy: 0.61

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Comparativa de Modelos

#### Modelo 1
- **Accuracy:** 0.7889  
- **Precision:** 0.7667  
- **Recall:** 0.6571  
- **F1 Score:** 0.7712  

**Características:**  
- Utiliza un Ansatz `StronglyEntanglingLayers` personalizado con capas definidas manualmente.  
- Optimización realizada con el algoritmo `AdamOptimizer`.  
- Presenta un mejor desempeño general en términos de precisión y balance entre las métricas.  
- El modelo logra una buena capacidad de predicción, con un equilibrio adecuado entre precisión y sensibilidad.  

**Observaciones:**  
Este modelo muestra un rendimiento sólido, siendo más confiable para clasificar correctamente los datos. Su precisión y F1 Score indican que es capaz de identificar correctamente las clases con un buen balance entre falsos positivos y falsos negativos.

---

#### Modelo 2
- **Accuracy:** 0.6111  
- **Precision:** 0.0  
- **Recall:** 0.0  
- **F1 Score:** 0.5057  

**Características:**  
- Utiliza el Ansatz `BasicEntanglerLayers`, que aplica rotaciones parametrizadas y puertas CNOT en una configuración estándar.  
- Optimización realizada con el algoritmo `SPSAOptimizer`.  
- Presenta un desempeño significativamente inferior en comparación con el Modelo 1.  

**Observaciones:**  
Este modelo tiene problemas para clasificar correctamente los datos, como lo indican los valores de precisión y recall en 0. Esto podría deberse a una configuración del Ansatz menos adecuada para el problema o a una convergencia insuficiente del optimizador SPSA.

---

### Conclusión
El **Modelo 1** es claramente superior en términos de desempeño, mostrando métricas más altas y un mejor balance entre precisión y sensibilidad. Esto sugiere que el Ansatz personalizado y el uso del optimizador AdamOptimizer son más efectivos para este conjunto de datos y problema específico. Por otro lado, el **Modelo 2** requiere ajustes en su configuración o entrenamiento para mejorar su capacidad predictiva.